In [1]:
from datasets import load_dataset
import datasets
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import torch
import numpy as np

In [3]:
# load dataset
ag_dataset = load_dataset('ag_news')

# Create a train/dev/test splits
ag_dev_dataset = load_dataset('ag_news', split='train[10%:11%]')
ag_train_dataset = load_dataset('ag_news', split='train[:10%]')
ag_test_dataset = load_dataset('ag_news', split='test[11%:12%]')

# merge the splits in a single `datasets.DatasetDict` object
ag_split = {split: data for split, data in zip(['train', 'test', 'dev'], [ag_train_dataset, ag_test_dataset, ag_dev_dataset])}
ag_dataset_split =  datasets.DatasetDict(ag_split)

# Count the number of labels.
# Important: use all the splits to compute the labels. 
num_labels = len(set(ag_dataset_split['train'].unique('label') + 
                     ag_dataset_split['test'].unique('label') +
                     ag_dataset_split['dev'].unique('label')))

Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


In [82]:
ag_dataset['train'].features['label'].names

['World', 'Sports', 'Business', 'Sci/Tech']

In [6]:
model_pretrained = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_pretrained)

"""🤔 **Understanding BERT tokenizer**"""

# Define a tokenization function for the dataset used a standard for text classification
def tokenize(dataset):
    sentences = dataset['text']
    return tokenizer(sentences, 
                     padding='max_length',
                     truncation=True)
    
# apply it 
ag_dataset_tokenized = ag_dataset_split.map(tokenize,
                                            batched=True,
                                            remove_columns=['text'],
                                            desc='Tokenize data')

Loading cached processed dataset at /home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-202c20a3ee59b4d0.arrow
Tokenize data: 100%|██████████| 1/1 [00:00<00:00, 13.46ba/s]
Loading cached processed dataset at /home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-0a9fc35bce44a8ca.arrow


# Assignment 1

In [62]:
# load model
from transformers import BertForSequenceClassification

checkpoint = 'bert-ag-news-classification'
model = BertForSequenceClassification.from_pretrained(checkpoint)

loading configuration file bert-ag-news-classification/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

In [32]:
test_data = ag_dataset_tokenized['test']
display(test_data)
print(type(test_data))

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 76
})

<class 'datasets.arrow_dataset.Dataset'>


In [31]:

X = test_data.pop['label']
y = test_data['label']



AttributeError: 'Dataset' object has no attribute 'pop'

In [20]:
test_trainer = Trainer(model) 

In [44]:
raw_pred = test_trainer.predict(test_data)
print(raw_pred) 

***** Running Prediction *****
  Num examples = 76
  Batch size = 8
69it [05:53,  3.07it/s]

PredictionOutput(predictions=array([[-2.5035717 , -2.9519646 ,  6.6624565 , -2.0078204 ],
       [-2.536739  ,  7.585553  , -2.2686212 , -2.2535117 ],
       [ 6.920326  , -3.1424928 , -1.7501704 , -2.3241503 ],
       [-2.6477304 , -3.7205415 ,  6.2286735 , -0.59329695],
       [-2.7142355 ,  7.541787  , -2.2311554 , -2.196851  ],
       [ 7.079184  , -3.0627112 , -2.202116  , -1.9797345 ],
       [-2.3463845 ,  7.655073  , -2.5317554 , -2.2617478 ],
       [-2.8457031 ,  7.572767  , -2.117573  , -2.1961508 ],
       [ 6.979641  , -2.9510245 , -1.8092418 , -2.5048544 ],
       [-2.296682  , -2.5306756 , -1.6837465 ,  6.9158545 ],
       [ 7.151075  , -3.085416  , -1.9056581 , -2.2572904 ],
       [ 6.5139275 , -2.826248  , -2.6685624 , -0.99780065],
       [ 7.106532  , -3.1863363 , -1.9037312 , -2.159581  ],
       [-1.1108124 ,  6.6217904 , -1.5569422 , -3.228916  ],
       [-1.2298164 , -2.609883  , -2.1834183 ,  6.597465  ],
       [-2.5323946 ,  7.5350814 , -2.3839374 , -2.2075  

In [47]:
raw_pred.predictions

array([[-2.5035717 , -2.9519646 ,  6.6624565 , -2.0078204 ],
       [-2.536739  ,  7.585553  , -2.2686212 , -2.2535117 ],
       [ 6.920326  , -3.1424928 , -1.7501704 , -2.3241503 ],
       [-2.6477304 , -3.7205415 ,  6.2286735 , -0.59329695],
       [-2.7142355 ,  7.541787  , -2.2311554 , -2.196851  ],
       [ 7.079184  , -3.0627112 , -2.202116  , -1.9797345 ],
       [-2.3463845 ,  7.655073  , -2.5317554 , -2.2617478 ],
       [-2.8457031 ,  7.572767  , -2.117573  , -2.1961508 ],
       [ 6.979641  , -2.9510245 , -1.8092418 , -2.5048544 ],
       [-2.296682  , -2.5306756 , -1.6837465 ,  6.9158545 ],
       [ 7.151075  , -3.085416  , -1.9056581 , -2.2572904 ],
       [ 6.5139275 , -2.826248  , -2.6685624 , -0.99780065],
       [ 7.106532  , -3.1863363 , -1.9037312 , -2.159581  ],
       [-1.1108124 ,  6.6217904 , -1.5569422 , -3.228916  ],
       [-1.2298164 , -2.609883  , -2.1834183 ,  6.597465  ],
       [-2.5323946 ,  7.5350814 , -2.3839374 , -2.2075    ],
       [ 2.8368506 , -3.

In [48]:
y_pred = np.argmax(raw_pred.predictions, axis=1)

In [38]:
y_true = test_data['label']
print(y_true)

[2, 1, 0, 3, 1, 0, 1, 1, 0, 3, 0, 0, 0, 1, 3, 1, 0, 1, 1, 1, 1, 1, 1, 3, 0, 2, 1, 1, 2, 2, 3, 2, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 3, 2, 0, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 0, 0, 2, 0, 0, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 3, 2]


# Confusion Matrix

In [49]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true, y_pred)

array([[18,  0,  0,  1],
       [ 0, 28,  0,  0],
       [ 1,  0, 17,  3],
       [ 0,  0,  1,  7]])

# Balanced Accuracy_score

In [50]:
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_true, y_pred)

0.9079730576441103

# Accuracy Score

In [51]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.9210526315789473

# f1_score

In [59]:
from sklearn.metrics import f1_score
f1_score_macro = f1_score(y_true, y_pred, average='macro').round(2)
f1_score_micro = f1_score(y_true, y_pred, average='micro').round(2)
f1_score_weighted = f1_score(y_true, y_pred, average='weighted').round(2)
print(f'f1_score_macro: {f1_score_macro}')
print(f'f1_score_micro: {f1_score_micro}')
print(f'f1_score_weighted: {f1_score_weighted}')

f1_score_macro: 0.89
f1_score_micro: 0.92
f1_score_weighted: 0.92


# recall

In [60]:
from sklearn.metrics import recall_score

recall_score_macro = recall_score(y_true, y_pred, average='macro').round(2)
recall_score_micro = recall_score(y_true, y_pred, average='micro').round(2)
recall_score_weighted = recall_score(y_true, y_pred, average='weighted').round(2)
print(f'recall_score_macro: {recall_score_macro}')
print(f'recall_score_micro: {recall_score_micro}')
print(f'recall_score_weighted: {recall_score_weighted}')

recall_score_macro: 0.91
recall_score_micro: 0.92
recall_score_weighted: 0.92


# Assignment 2

In [63]:
from transformers import BertForSequenceClassification

checkpoint = 'bert-ag-news-classification'
model = BertForSequenceClassification.from_pretrained(checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_pretrained)

loading configuration file bert-ag-news-classification/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

In [72]:
from transformers import TextClassificationPipeline
pipeline_classify = TextClassificationPipeline(model=model, tokenizer=tokenizer)

## World = 0
## Sports = 1
## Business = 2
## Sci/Tech = 3


In [84]:
while True:
    prompt = input('\n\nInsert prompt\n')
    if prompt == 'exit':
        break
    else:
        print(pipeline_classify(prompt))



[{'label': 'LABEL_2', 'score': 0.9976270794868469}]
